In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from scipy.stats import uniform, randint
from feature_hasher_transformer import FeatureHasherTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from feature_hasher_transformer import FeatureHasherTransformer
import joblib


# 1) LOAD + PREPROCESS TRAIN DATA
df = pd.read_csv("data/train/cleaned_data.csv")

# (a) Drop any row where material_price is missing or ≤0
df = df.dropna(subset=['material_price'])
df = df[df['material_price'] > 0]

# (b) Fill missing text columns (including surname and action)
text_columns_train = [
    'material_name',
    'material_type',
    'material_subtype',
    'surgeon_name',
    'surgeon_surname',
    'procedure_name'
]
df[text_columns_train] = df[text_columns_train].fillna('missing').astype(str)

# We also assume there is a column called 'surgeon_specific_action' in cleaned_data.csv,
# whose values are "ADDED" or "DEFAULT". Fill any missing with 'missing' too:
df['surgeon_specific_action'] = df['surgeon_specific_action'].fillna('missing').astype(str)

# (c) Create combined “surgeon_fullname” from name + surname
df['surgeon_fullname'] = (
    df['surgeon_name'].str.strip() + ' ' + df['surgeon_surname'].str.strip()
)

# (d) Normalize material_name to lowercase to match test_data
df['material_name'] = df['material_name'].str.lower()

# (e) Combine text features for hashing
df['combined_features'] = df[
    ['material_name', 
     'material_type', 
     'material_subtype',
     'surgeon_fullname', 
     'procedure_name']
].agg(' '.join, axis=1)

# (f) Log-transform the target
y_log = np.log(df['material_price'].values)


# 2) BUILD LOOKUP: (material_name, surgeon_fullname) → (material_price, surgeon_specific_action)
# Keep only the first unique (material_name, surgeon_fullname)––that row’s material_price + action
lookup = (
    df[['material_name', 'surgeon_fullname', 'material_price', 'surgeon_specific_action']]
      .drop_duplicates(subset=['material_name', 'surgeon_fullname'], keep='first')
      .rename(columns={
          'surgeon_fullname':          'train_surgeon_fullname',
          'material_price':            'train_material_price',
          'surgeon_specific_action':   'train_surgeon_specific_action'
      })
)
# Now `lookup` has columns:
#    [material_name, train_surgeon_fullname, train_material_price, train_surgeon_specific_action]

# 3) TRAIN/TEST SPLIT for the MODEL
X_train_text, X_test_text, y_train_log, y_test_log = train_test_split(
    df['combined_features'], 
    y_log, 
    test_size=0.2, 
    random_state=42
)

pipeline = Pipeline([
    ('hasher', FeatureHasherTransformer(n_features=100)),
    ('model', XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42
    ))
])

param_distributions = {
    'model__n_estimators':     randint(50, 300),
    'model__max_depth':        randint(3, 10),
    'model__learning_rate':    uniform(0.01, 0.3),
    'model__subsample':        uniform(0.5, 0.5),
    'model__colsample_bytree': uniform(0.5, 0.5),
    'model__reg_alpha':        uniform(0, 1),
    'model__reg_lambda':       uniform(0, 1),
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=30,
    scoring='r2',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train_text, y_train_log)
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)


# 4) OPTIMIZATION ON TEST SET + MERGE LOOKUP
# (a) Build dict of material → min price (for get_cheapest_price)
historical_min_prices = df.groupby('material_name')['material_price'].min().to_dict()

# (b) Load test_data
test_data = pd.read_csv('data/test/new_data.csv', low_memory=False)

# (c) Fill missing test‐side text columns (including surname and action)
text_columns_test = [
    'material_name', 
    'material_type', 
    'material_subtype',
    'surgeon_name',   
    'surgeon_surname', 
    'procedure_name'
]
test_data[text_columns_test] = test_data[text_columns_test].fillna('missing').astype(str)

# We assume test_data also has a column 'surgeon_specific_action';
# if not, you can drop or rename accordingly:
test_data['surgeon_specific_action'] = test_data['surgeon_specific_action'].fillna('missing').astype(str)

# (d) Create combined “surgeon_fullname” in test_data
test_data['surgeon_fullname'] = (
    test_data['surgeon_name'].str.strip() + ' ' + test_data['surgeon_surname'].str.strip()
)

# (e) Normalize material_name
test_data['material_name'] = test_data['material_name'].str.lower()

# (f) Rebuild combined_features for hashing
test_data['combined_features'] = test_data[
    ['material_name', 
     'material_type', 
     'material_subtype',
     'surgeon_fullname', 
     'procedure_name']
].agg(' '.join, axis=1)

# (g) Create is_default flag exactly as before
test_data['is_default'] = (test_data['surgeon_name'] == 'Standardized').astype(int)

# (h) Ensure no new material is missing from historical_min_prices
missing_materials = set(test_data['material_name']) - set(historical_min_prices.keys())
if missing_materials:
    raise ValueError(f"Materials {missing_materials} not found in cleaned_data.csv")

# (i) Map the “min‐price” for optimization logic
test_data['material_price'] = test_data['material_name'].map(historical_min_prices)

# (j) Merge in the “original” train‐file info (price + action) 
#     by matching on (material_name, surgeon_fullname)
test_data = test_data.merge(
    lookup,
    left_on=['material_name', 'surgeon_fullname'],
    right_on=['material_name', 'train_surgeon_fullname'],
    how='left'
)
# Now test_data has columns:
#    … , 'train_surgeon_fullname', 'train_material_price', 'train_surgeon_specific_action', …

# (k) Predict & back‐transform
log_preds = best_model.predict(test_data['combined_features'])
test_data['predicted_price'] = np.exp(log_preds)

# (l) Ensure predicted_price ≥ overall train‐min
min_train_price = df['material_price'].min()
test_data['predicted_price'] = np.clip(test_data['predicted_price'], min_train_price, None)

# (m) Helper to pick the “cheapest” (min from df)
def get_cheapest_price(row):
    hist_price = row['material_price']  # this is the MIN from df
    if not np.isnan(hist_price):
        return hist_price, 'historical'
    else:
        raise ValueError(f"No historical price found for material {row['material_name']}")

# (n) Build final results
results = []
for proc_id in test_data['procedure_id'].unique():
    proc_data = test_data[test_data['procedure_id'] == proc_id].copy()

    # Identify default and surgeon-added materials
    default_data = proc_data[proc_data['is_default'] == 1]
    default_materials = set(default_data['material_name'])
    surgeon_data = proc_data[proc_data['is_default'] == 0]
    surgeon_added = set(
        surgeon_data[surgeon_data['surgeon_specific_action'] != 'default']['material_name']
    ) - default_materials
    all_materials = default_materials.union(surgeon_added)

    # Compute original total cost for the procedure
    orig_prices_list = []
    for mat in all_materials:
        rows = proc_data[proc_data['material_name'] == mat]
        if not rows.empty:
            orig_prices_list.append(rows.iloc[0]['train_material_price'])
    procedure_original_cost = sum(orig_prices_list)

    # Gather optimized materials info
    optimized_materials = {}
    for mat in all_materials:
        rows = proc_data[proc_data['material_name'] == mat]
        if rows.empty:
            continue
        row0 = rows.iloc[0]
        # Get cheapest historical price
        cheapest_price = historical_min_prices.get(mat, np.nan)
        # Collect new fields
        material_id = row0.get('material_id')
        material_type = row0.get('material_type')
        material_subtype = row0.get('material_subtype')
        specialty = row0.get('speciality')  # rename to 'specialty'
        procedure_name = row0.get('procedure_name')
        # Original train info
        surgeon_fullname = row0['train_surgeon_fullname']
        surgeon_specific_action = row0['train_surgeon_specific_action']

        optimized_materials[mat] = {
            'material_id': material_id,
            'material_name': mat,
            'material_original_price': row0['train_material_price'],
            'material_optimized_price': cheapest_price,
            'material_type': material_type,
            'material_subtype': material_subtype,
            'specialty': specialty,
            'procedure_id': proc_id,
            'procedure_name': procedure_name,
            'procedure_original_cost': procedure_original_cost,
            'price_source': 'historical' if not np.isnan(cheapest_price) else 'none',
            'surgeon_fullname': surgeon_fullname,
            'surgeon_specific_action': surgeon_specific_action
        }

    # Compute optimized cost for the procedure
    procedure_optimized_cost = sum(
        info['material_optimized_price'] for info in optimized_materials.values() if not np.isnan(info['material_optimized_price'])
    )

    # Append to results with final cost assigned
    for mat, info in optimized_materials.items():
        row = info.copy()
        # add procedure_optimized_cost
        row['procedure_optimized_cost'] = procedure_optimized_cost
        results.append(row)

# Convert to DataFrame and reorder columns
columns_order = [
    'material_id', 'material_name', 'material_original_price', 'material_optimized_price',
    'material_type', 'material_subtype', 'specialty', 'procedure_id', 'procedure_name',
    'procedure_original_cost', 'procedure_optimized_cost', 'price_source',
    'surgeon_fullname', 'surgeon_specific_action'
]
results_df = pd.DataFrame(results)[columns_order]

# Save to CSV
results_df.to_csv('models/optimized_results.csv', index=False)

# Save lookup and model as before
joblib.dump(lookup, 'models/lookup.joblib')
joblib.dump(historical_min_prices, 'models/min_prices.joblib')
joblib.dump(best_model, 'models/model_weights.joblib')

print('Results saved to models/optimized_results.csv')


C:\Users\HP\AppData\Local\Temp\ipykernel_29248\3647235274.py:15: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/train/cleaned_data.csv")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Hyperparameters: {'model__colsample_bytree': np.float64(0.9244569121330419), 'model__learning_rate': np.float64(0.22651885634946198), 'model__max_depth': 9, 'model__n_estimators': 191, 'model__reg_alpha': np.float64(0.49379559636439074), 'model__reg_lambda': np.float64(0.5227328293819941), 'model__subsample': np.float64(0.7137705091792748)}
Results saved to models/optimized_results.csv
